<a href="https://colab.research.google.com/github/KshitijS093/Mini-Project-2/blob/main/Mini_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
mkdir model
mkdir data

SyntaxError: invalid syntax (<ipython-input-26-18e51f666fea>, line 1)

In [27]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

In [28]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from transformers import BertForSequenceClassification, BertTokenizer

# Define the cache directory where model checkpoints will be saved
cache_dir = "/content/model"  # Change this to the desired cache directory

# Load the pre-trained BERT model with caching enabled
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', cache_dir=cache_dir)

# Access the model's configuration to get the cache directory
cache_dir = model.config._cache_dir
print(cache_dir)



config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: 'BertConfig' object has no attribute '_cache_dir'

In [29]:
# Load the CSV files into pandas DataFrames
train_df = pd.read_csv('/content/data/train_14k_split.csv')
val_df = pd.read_csv('/content/data/val_3k_split.csv')
test_df = pd.read_csv('/content/data/test_unalbelled.csv')

# Optional: Display the first few rows of each DataFrame to understand the data structure
print(train_df.head())
print(val_df.head())
print(test_df.head())

# Assuming your data has 'text' column for input text and 'label' column for sentiment labels
train_texts = train_df['sentence'].tolist()
train_labels = train_df['label'].tolist()
val_texts = val_df['sentence'].tolist()
val_labels = val_df['label'].tolist()
test_texts = test_df['sentence'].tolist()

# # Convert labels to integers if needed (e.g., 'positive': 1, 'negative': 0)
# label_map = {'positive': 1, 'negative': 0}
# train_labels = [label_map[label] for label in train_labels]
# val_labels = [label_map[label] for label in val_labels]


      id                                           sentence  label sentiment
0   4330  nen á vist bolest vztek smutek zmatek osam ě l...      1   neutral
1  41616  Haan yaar neha pensive pensive kab karega woh ...      1   neutral
2   6648  television media congress ke liye nhi h Ye toh...      0  negative
3   2512  All India me nrc lagu kare w Kashmir se dhara ...      2  positive
4    610  who Pagal hai kya They aren t real issues Mand...      1   neutral
      id                                           sentence  label sentiment
0  30258  modi mantrimandal may samil honay par badhai n...      2  positive
1  16648                Rashid Tu toh naamakool hai Mare h       0  negative
2  28511  U saw caste and religion in them nation saw ta...      0  negative
3  10466  sir local police station pe complaint krne par...      1   neutral
4  19266  Ve Maahi song from Kesari is current favourite...      2  positive
      id                                           sentence
0  20803        

In [30]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)


In [31]:
class UnlabeledSentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Create the test dataset without labels
unlabeled_test_dataset = UnlabeledSentimentDataset(test_encodings)

In [19]:
from transformers import BertForSequenceClassification

# Load the trained model
model = BertForSequenceClassification.from_pretrained('/content/model/models--bert-base-uncased')


OSError: /content/model/models--bert-base-uncased does not appear to have a file named config.json. Checkout 'https://huggingface.co//content/model/models--bert-base-uncased/tree/main' for available files.

In [10]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)


NameError: name 'test_labels' is not defined

In [32]:
import torch

def predict_sentiment(model, dataset):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=16)
    with torch.no_grad():
        for batch in dataloader:
            inputs = {key: batch[key].to(model.device) for key in batch}
            outputs = model(**inputs)
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
    return predictions


In [39]:
# Make predictions on the unlabeled test dataset
test_predictions = predict_sentiment(model, unlabeled_test_dataset)


KeyboardInterrupt: 

In [40]:
print(test_predictions)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [42]:
test_predictions.count(1)

2990